In [1]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np
import pandas as pd

sys.path.append("/home/redha/prisma_chap4/prisma/source/auxilliaries")
from utils import convert_tb_data


def plot_metric_vs_sync_step(result_df, exps_name, metric_name, axis):
    # plot the metric vs. sync step given a dataframes of results
    data = result_df[result_df.name == metric_name].sort_values("step")
    x = data.step.values
    y = data.value.values
    axis.plot(x, y, label=exps_name)


# The directory where the test results are stored
dir_path = "/home/redha/prisma_chap4/prisma/examples/5n_overlay_full_mesh_abilene/results/Final_results"
# dir_path = '/mnt/journal_paper_results/ex/5n_overlay_full_mesh_abilene/results/Final_results'
dir_path = "/mnt/backup_examples_new/5n_overlay_full_mesh_abilene/results/abilene_results_with_threshold"
dir_path = "/mnt/backup_examples_new/overlay_full_mesh_10n_geant/results/geant_results_with_threshold"
# dir_path = '/mnt/final_results_geant'
dir_path = "/home/redha/prisma_chap4/prisma/examples/abilene/results/abilene_results"
plot_style = {
    "font.family": "Times New Roman",
    "font.size": 50,
    "font.weight": "bold",
    "axes.labelweight": "bold",
    "axes.titleweight": "bold",
    "axes.linewidth": 3,
    "axes.grid": True,
    "axes.grid.which": "both",
    "axes.spines.left": False,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.spines.bottom": False,
    "axes.facecolor": "#EBEBEB",
    "xtick.color": "black",
    "ytick.color": "black",
    "ytick.minor.visible": False,
    "xtick.minor.visible": False,
    "grid.color": "black",
    "grid.linestyle": "-.",
    "grid.linewidth": 0.6,
    "figure.autolayout": True,
    "figure.figsize": (19.4, 10),
}

plt.rcParams.update(plot_style)
# evaluation metric name
metric_names = {
    "stats": [
        "avg_hops_over_time",
        "signalling ratio",
        "total_hops_over_time",
        "all_signalling_ratios",
    ],
    "nb_new_pkts": ["pkts_over_time"],
    "nb_lost_pkts": ["pkts_over_time"],
    "nb_arrived_pkts": ["pkts_over_time"],
    "test_results": [
        "test_overlay_loss_rate",
        "test_overlay_e2e_delay",
        #  "test_global_loss_rate",
        #  "test_global_e2e_delay"
    ],
}
metric_name = "test_overlay_loss_rate"
exps_names = []

# ------------------------------------------------------------
# % set the parameters
# static parameters
traff_mats = [0, 2, 3, 1]
# traff_mats = list(range(11))
traff_mats = [0,]
sync_steps = list(range(1, 8))
sync_steps = list(range(1, 10)) + list(range(10, 22, 2))
sync_steps = list(range(1, 10, 2))
# sync_steps.remove(1)
# sync_steps.remove(3)
sync_steps = [1,]
seed = 100
rb_size = 10000
signaling_types = ["dqn_logit_sharing"]
# signaling_types = ["dqn_model_sharing", "sp", "opt_10"]
# signaling_types = ["dqn_model_sharing",]
# dqn_models = ["", "_lite", "_lighter", "_lighter_2", "_lighter_3", "_ff"][:-1]
dqn_models = [
    "",
]
nn_size = 35328
# variable parameters
training_loads = [0.4, 0.7, 0.9]
training_loads = [0.4]
topology_name = "abilene"
bs = 512
lr = 0.0001
exploration = ["vary", 1.0, 0.01]
ping_freq = 10000
mv_avg_interval = 5
max_output_buffer_size = 16260
train_duration = 20
test_loads = [0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2]
loss_penalty_types = ["fixed", "None"]
name_to_save = [ "None"]
lambda_train_step = -1
buffer_soft_limit = 0
snapshots = ["final"]
# snapshots = ["final"] + [f"episode_{i}_step_{i}" for i in range(1, 20, 1)]
ping_freqs = [0.1, 0.5, 1]
ping_freqs = [0.1,]
# snapshots =  ["final"]

# % reset the variables
all_exps_results = {}
# go through all the experiments
found = []
thresholds = [0.0, 0.25, 0.5, 0.75]
thresholds = [0.0]
loss_flag = 1
use_loss_flag = 1
optimal_root_path = (
    "/mnt/backup_examples_new/5n_overlay_full_mesh_abilene/optimal_solution/"
)


def load_data(path, metric_name, case):
    exp_df = convert_tb_data(path)
    if case in ["stats", "nb_new_pkts", "nb_lost_pkts", "nb_arrived_pkts"]:
        if metric_name in ("all_signalling_ratios",):
            # compute average signalling ratio per episode
            exp_df.step = pd.to_datetime(exp_df.step / 1000, unit="ms")
            exp_df = exp_df.resample("20s", on="step", closed="right").mean()
            exp_df.step = exp_df.index
        else:
            exp_df = exp_df.iloc[-1]
    steps, indices = np.unique(exp_df.step, return_index=True)
    if len(steps) > 1:
        np.array(exp_df.value)[indices].reshape(-1, 1)
    else:
        np.array(exp_df.value).reshape(-1, 1)


# ------------------------------------------------------------

2024-04-07 11:43:35.682472: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 11:43:35.730137: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 11:43:36.145881: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 11:43:36.148800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 11:43:38.116788: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [25]:
# Plan B: plot overhead vs loss rate and delay for different thresholds and sync step 5 for model sharing, logit sharing, and value sharing for mat 0
# fig1, ax1 = plt.subplots()
# fig2, ax2 = plt.subplots()
sync_step = 5
for signalling in [ "dqn_model_sharing", "dqn_logit_sharing", "dqn_value_sharing"]:
    for threshold in [0.0,]:
        if signalling == "dqn_value_sharing":
            sync_step = 1
        else:
            sync_step = 5
        path = f"/home/redha/prisma_chap4/prisma/examples/abilene/results/abilene_results/sync_{sync_step}_mat_0_dqn__{signalling}_size_35328_tr_0.4_sim_60_1_lr_0.0001_bs_512_outb_16260_losspen_fixed_vary_reset_0_threshold_{threshold}/"
        if signalling == "dqn_model_sharing":
            path = path.rstrip("/") + "_off/"
        try:
            df = convert_tb_data(path + "test_results")
        except Exception as e:
            print("not found", signalling, sync_step, path)
            continue
        loss = df[df.name == "final_global_loss_rate_per_load"]
        delay = df[df.name == "final_global_e2e_delay_per_load"]
        indices, order = np.unique(loss.step, return_index=True)
        loss_y = np.array(loss.value)[order]
        delay_y = np.array(delay.value)[order]
        print("loss", signalling, sync_step, loss_y)
        print("delay", signalling, sync_step, delay_y)
        try:
            df = convert_tb_data(path + "stats")
        except Exception as e:
            print("not found", signalling, sync_step, path)
            continue
        overhead = df[df.name=="sim_signaling_overhead_per_time"]["value"].to_numpy()[-1]
        print("overhead", signalling, sync_step, overhead)



loss dqn_model_sharing 5 [0.         0.01745989 0.04747862 0.06453644 0.09589881 0.1304086
 0.16966967]
delay dqn_model_sharing 5 [0.04534    0.069475   0.088844   0.122656   0.160171   0.18908601
 0.194628  ]
not found dqn_model_sharing 5 /home/redha/prisma_chap4/prisma/examples/abilene/results/abilene_results/sync_5_mat_0_dqn__dqn_model_sharing_size_35328_tr_0.4_sim_60_1_lr_0.0001_bs_512_outb_16260_losspen_fixed_vary_reset_0_threshold_0.0_off/
loss dqn_logit_sharing 5 [0.         0.         0.01810675 0.05004726 0.09304081 0.12908223
 0.16484819]
delay dqn_logit_sharing 5 [0.033115   0.042462   0.103927   0.14484701 0.16966    0.19339199
 0.21712901]
not found dqn_logit_sharing 5 /home/redha/prisma_chap4/prisma/examples/abilene/results/abilene_results/sync_5_mat_0_dqn__dqn_logit_sharing_size_35328_tr_0.4_sim_60_1_lr_0.0001_bs_512_outb_16260_losspen_fixed_vary_reset_0_threshold_0.0/
loss dqn_value_sharing 1 [0.71032089]
delay dqn_value_sharing 1 [0.42557099]
not found dqn_value_sharin

In [26]:
convert_tb_data(path + "stats")

KeyError: "None of [Index(['wall_time', 'name', 'step', 'value'], dtype='object')] are in the [columns]"